<a href="https://colab.research.google.com/github/ismailtarik/Analyse_Bibliometrique_des_Publications_Scientifiques_avec_API_Scopus/blob/main/entrainement_fasttext_%26_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall spacy -y
!pip install spacy
!pip install gensim
!pip install textBlob
!pip install fasttext
!python -m spacy download en_core_web_lg
import os
os.kill(os.getpid(), 9)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/1-3m-linkedin-jobs-and-skills-2024")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd

# Update the path to include the actual CSV file
file_path = '/root/.cache/kagglehub/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024/versions/2/job_summary.csv'

In [ ]:
job_descriptions = pd.read_csv(file_path)
job_descriptions.head()
job_descriptions = job_descriptions[:300000]
job_descriptions.to_csv("job_descriptions.csv", index=False)

In [ ]:
job_descriptions = pd.read_csv("/content/job_descriptions.csv")
job_descriptions = job_descriptions[['job_summary']]
job_descriptions.head()

In [ ]:
import pandas as pd
skills_dict = pd.read_csv("/content/competences_dictionnaires_finales.csv")
skills_dict.head()

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')

# Fonction de nettoyage de texte
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words('english') and t not in string.punctuation]
    return " ".join(tokens)

# Prétraitement des descriptions
job_descriptions['job_summary'] = job_descriptions['job_summary'].apply(preprocess_text)
job_descriptions.to_csv("job_descriptions.csv", index=False)

# Prétraitement des compétences
skills_dict = pd.read_csv("/content/competences_dictionnaires_finales.csv")

# Appliquer split ligne à ligne (si les compétences sont séparées par \n dans une cellule)
skills_dict['skills_keywords'] = skills_dict['skills_keywords'].apply(lambda x: [s.strip().lower() for s in str(x).split('\n')])

# Convertir en liste plate
skills_dict = skills_dict.explode('skills_keywords').reset_index(drop=True)

skills_dict.to_csv("/content/competences_dictionnaires_finales.csv", index=False)

print(skills_dict.head())

In [ ]:
import pandas as pd

def create_corpus_from_csv(csv_path, output_path="corpus.txt"):
    descriptions = job_descriptions['job_summary'].dropna().tolist()

    with open(output_path, 'w', encoding='utf-8') as f:
        for desc in descriptions:
            desc = desc.replace('\n', ' ').strip()
            f.write(desc + '\n')

    print(f"Corpus créé avec {len(descriptions)} offres dans {output_path}")

create_corpus_from_csv("/content/job_descriptions.csv")

In [ ]:
from gensim.models import FastText
from nltk.tokenize import word_tokenize
import nltk
import string

nltk.download('punkt')
nltk.download('punkt_tab')

# Charger ton corpus
def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    tokenized = []
    for line in lines:
        line = line.lower()
        line = line.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(line, language='english')
        tokenized.append(tokens)
    return tokenized

# Entraîner le modèle FastText
def train_fasttext_model(tokenized_corpus, model_path="fasttext_model.bin"):
    model = FastText(sentences=tokenized_corpus, vector_size=100, window=5, min_count=2, workers=4, epochs=10)
    model.save(model_path)
    print(f"Modèle entraîné et enregistré sous : {model_path}")

if __name__ == "__main__":
    corpus = load_corpus("/content/corpus.txt")
    train_fasttext_model(corpus)

In [ ]:
def train_doc2vec_model(job_descriptions):
    """ Entraînement du modèle Doc2Vec """
    documents = [TaggedDocument(words=preprocess_text(desc), tags=[str(i)]) for i, desc in enumerate(job_descriptions)]
    model_d2v = Doc2Vec(vector_size=100, window=5, min_count=1, workers=2, epochs=20, dm=0)
    model_d2v.build_vocab(documents)
    model_d2v.train(documents, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs)
    model_d2v.save("doc2vec_jobs.model")
    return model_d2v

In [ ]:
model_d2v = train_doc2vec_model(job_descriptions)